In [1]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
all_data_list = os.listdir('/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/testcase/testcases')
data_num = len(all_data_list)
print(data_num)
datadir = '/Users/wangyanpu/Desktop/台灣大學/大三下/作業研究/case/Case2/testcase/testcases'
instances = []
for i in range(len(all_data_list)):
    name = all_data_list[i]
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath,encoding='unicode_escape'))

188


In [3]:
# job structure 
class Job:
    '''structure for 1 job '''
    def __init__(self, row):
        '''input := df.iloc[idx, :]'''
        self.id = row['Job ID']+1
        self.due = row['Due Time']
        self.stage_states = [False for _ in range(2)] # True as complete, False as not yet processed
        self.stage_pt = [row['Stage-1 Processing Time'], row['Stage-2 Processing Time']]
        mfor1 = list(map(int, row['Stage-1 Machines'].split(',')))
        if row['Stage-2 Machines'] is not np.nan:
            mfor2 = list(map(int, row['Stage-2 Machines'].split(',')))
        else: mfor2 = [] 
        self.stage_mach = [mfor1, mfor2]
        self.assign_mach = [None for _ in range(2)]
        self.start_time = [-1 for _ in range(2)]
        self.end_time = [-1 for _ in range(2)]
    
    def __repr__(self):
        return f'\
          * Job id: {self.id}\n\
          * Due time:{self.due}\n\
          stage 1: {self.stage_states[0]}, {self.assign_mach[0]}\n\
                   {self.stage_pt[0]}, {self.stage_mach[0]}\n\
          stage 2: {self.stage_states[1]}, {self.assign_mach[1]}\n\
                   {self.stage_pt[1]}, {self.stage_mach[1]}'
    __str__ = __repr__

In [4]:
class Jobs:
    '''structure for multiple jobs' management'''
    def __init__(self, n):
        self.completion_times = np.zeros(n)
        self.tardiness = np.zeros(n)
    def get_RRDD(self):
        if getattr(self, 'RRDD', None) is None:
            self.RRDD = self.due_dates - np.min(self.due_dates)
        return self.RRDD # static
    
    def add_jobs(self, datas):
        self.due_dates = df['Due Time'].to_numpy()
        self.jobs = []
        for i in range(len(datas)):
            row = datas.iloc[i, :]
            jobi = Job(row)
            self.jobs.append(jobi)
    
    def assign(self, job_name, mach, st):
        '''job_name = (2, 0) means job 3 and op 1
        note that job and op is 0-indexed as well as machines
        op
        '''
        
        i = 0 
        jobidx, op = job_name 
        job = self.jobs[jobidx]
        while i < op:
            if job.stage_states[i] is not True:
                print(f'Error scheduling operation: previous operation {i} hasn\'t been scheduled.')
                return 
            i += 1
        J.completion_times[jobidx] = st + job.stage_pt[op]
        job.assign_mach[op] = mach
        job.start_time[op] = st
        job.end_time[op] = J.completion_times[jobidx]
        job.stage_states[op] = True 

In [5]:
class Machines:
    def __init__(self, df):
        '''pass the stage1, stage2 machine lists'''
        mfor1 = df['Stage-1 Machines'].values.tolist()
        mfor2 = df['Stage-2 Machines'].values.tolist()
        mfor1 = [list(map(int, x.split(','))) for x in mfor1]
        mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
        mfor1 = sum(mfor1, [])
        mfor2 = sum(mfor2, [])
        self.number = max(max(mfor1), max(mfor2))
        self.versatile = [mfor1.count(i+1) + mfor2.count(i+1) for i in range(self.number)]
        self.schedule = [[] for _ in range(self.number)]
        self.span = [[] for _ in range(self.number)]
        self.fintime = [0 for _ in range(self.number)]
        
    def is_available(self):
        '''pass machine 編號(1-indexed) to get the state'''
        pass
    def _schedule(self, mach, job_name, proc_time):
        '''mach is 0-indexed'''
        display_name = tuple([x+1 for x in job_name])
        self.schedule[mach].append(display_name) 
        self.span[mach].append(proc_time)
        self.fintime[mach] += proc_time

In [11]:
Total_tardy = []
Total_makespan = []
error_inst = []
count = 0

for inst in range(data_num):
    
    try:
        df = instances[inst]
        due_dates = df['Due Time'].to_numpy()
        M = Machines(df)
        M.versatile
        J = Jobs(len(df))
        J.add_jobs(df)
        # for the first operation
        temp_completion_times = np.array([job.stage_pt[0] for i, job in enumerate(J.jobs)])
        temp_completion_times

        order = np.where(temp_completion_times)# gives stable sort
        # the job index order (0-indexed) to be assigned 
        np.random.shuffle(order[0])
        # schedule the first operation in accordance to tardiness

        # which index
        batch_index = 0

        for jidx in order[0]:
            job = J.jobs[jidx]
            job_name = (jidx, batch_index)
            job_proc_time = job.stage_pt[batch_index]
            if job_proc_time <= 0:
                # update job
                J.assign(job_name = job_name, 
                        mach = -1,
                        st = job.end_time[batch_index-1]) 
                # note that it's only possible for second operation to have proc time = 0
                # so this doesn't trigger index error
                continue
            # note that the available machines here is 1-indexed, change them to 0-indexed
            avail_machines_idx = [x-1 for x in job.stage_mach[batch_index]]
            # The less versatile, the less fintime, the better. Break ties with smallest-index rule.
            curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x],M.versatile[x], x))
            # schedule the operation on curr_machine
            M._schedule(job_name = job_name, 
                       mach = curr_machine, 
                       proc_time = job_proc_time)
            # update the Jobs objects
            # start time is either concatenated to the machines's last scheduled operation's end
            # or the end of its previous operation 
            J.assign(job_name = job_name, 
                    mach = curr_machine, 
                     st = max(M.fintime[curr_machine], J.completion_times[jidx])
                    )

        batch_index = 1
        for jidx in order[0]:
            job = J.jobs[jidx]
            job_name = (jidx, batch_index)
            job_proc_time = job.stage_pt[batch_index]
            if job_proc_time <= 0:
                # update job
                J.assign(job_name = job_name, 
                        mach = -1,
                        st = job.end_time[batch_index-1]) 
                # note that it's only possible for second operation to have proc time = 0
                # so this doesn't trigger index error
                continue
            # note that the available machines here is 1-indexed, change them to 0-indexed
            avail_machines_idx = [x-1 for x in job.stage_mach[batch_index]]
            # The less versatile, the less fintime, the better. Break ties with smallest-index rule.
            curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x],M.versatile[x], x))
            # schedule the operation on curr_machine
            M._schedule(job_name = job_name, 
                       mach = curr_machine, 
                       proc_time = job_proc_time)
            # update the Jobs objects
            # start time is either concatenated to the machines's last scheduled operation's end
            # or the end of its previous operation 
            J.assign(job_name = job_name, 
                    mach = curr_machine, 
                     st = max(M.fintime[curr_machine], J.completion_times[jidx])
                    )

        print(f'** Summary \ninstance {inst+1}:')
        Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
        Tardy_jobs = [x+1 for x in Tardy_jobs]
        Makespan = max(M.fintime)
        print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
        print('Second objective (makespan):', Makespan)
        print('==================================')

        Total_tardy.append(len(Tardy_jobs))
        Total_makespan.append(Makespan)
        count += 1
        
    except:
        print("error!")
        error_inst.append(inst)

print(len(Total_makespan))
print(len(Total_tardy))
average_tardy = sum(Total_tardy)/count
average_makespan = sum(Total_makespan)/count
print("cases with error:", *error_inst)
print('Random Easy cases (%d instances):' %(count))
print("average tardy job for all these cases "+str(average_tardy))
print("average makespans for all these cases "+str(average_makespan))

** Summary 
instance 1:
First objective (# tardy): 15 [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 16, 17, 18]
Second objective (makespan): 25.2
** Summary 
instance 2:
First objective (# tardy): 2 [1, 4]
Second objective (makespan): 7.5
** Summary 
instance 3:
First objective (# tardy): 11 [1, 2, 3, 6, 8, 9, 10, 11, 12, 14, 15]
Second objective (makespan): 18.4
** Summary 
instance 4:
First objective (# tardy): 20 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23]
Second objective (makespan): 27.5
** Summary 
instance 5:
First objective (# tardy): 4 [1, 3, 6, 7]
Second objective (makespan): 16.200000000000003
** Summary 
instance 6:
First objective (# tardy): 1 [1]
Second objective (makespan): 5.6
** Summary 
instance 7:
First objective (# tardy): 3 [1, 2, 4]
Second objective (makespan): 15.100000000000001
** Summary 
instance 8:
First objective (# tardy): 14 [2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18]
Second objective (makespan): 16.5
** Summary 
instance 9:


** Summary 
instance 158:
First objective (# tardy): 27 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 30]
Second objective (makespan): 19.9
** Summary 
instance 159:
First objective (# tardy): 1 [1]
Second objective (makespan): 8.2
** Summary 
instance 160:
First objective (# tardy): 14 [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16]
Second objective (makespan): 20.2
** Summary 
instance 161:
First objective (# tardy): 34 [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 39]
Second objective (makespan): 26.099999999999998
** Summary 
instance 162:
First objective (# tardy): 1 [1]
Second objective (makespan): 6.9
** Summary 
instance 163:
First objective (# tardy): 49 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Second obj

In [31]:
print(12.823529411764707/137.7058823529412)
print(24.82085561497325/286.89946524064584)

0.093122597180692
0.08651412296692149
